This file defines a basic construction game, inspired in the exchange phase of Catan. There are 3 players, at each turn one of the player receives NRPP resources of NR types. With these resources they can then make some constructions. There are two types of constructions either [2,1] (two resources of the first type and one of the second) or [1,2]. They can exchange resources with the other players.   

The class Game defines the rules of the game and provide an infrastructure to run a game. The class Player defines the player behavior in terms of proposals it makes and proposals it accepts. The function propose defines the proposals to other players, and the function answerProposal defines if the player accepts or not an exchange. Three behaviors are already defined 'yesman' that always accepts proposals, 'fair' that only accepts fair or for profit exchanges and 'profit' that only accepts exchanges if they make a profit.

A player can make multiple proposal but only the first one accepted is executed. So the player needs to provide the proposals in the order of priority.

In the lab you can explore different strategies for the player. For instance collusion, take into consideration the points of each players, or others. For the project more complex strategies would need to be explored for instance long-term thinking, adaptative methods.

**This is just a skeleton to explore negotiation in a semi-cooperative game. The code itself is the documentation, and it is not bug free. The rules of the game can be changed if it provides opportunities to study other approaches.**

In [8]:
import numpy as np
import matplotlib.pyplot as plt

class Player:
    # policies
    # yesman - always accepts proposals
    # fair   - only fairorprofit
    # profit - only profit
    def __init__(self, pid, NR, NP, recipes, policy = 'yesman'):
        self.pid = pid
        self.NR = NR
        self.res = np.zeros(self.NR)
        self.otherplayer = []
        self.recipes = recipes
        self.policy = policy
        for ii in range(NP):
            if ii == pid:
                continue
            self.otherplayer.append( ii )

    def checkifpossibleexchange(self, resourcespplayer, exchanges):
        # print("checkifpossibleexchange", exchanges)
        filteredexchanges = []
        for proposedto in exchanges:
            # print("proposedto ", proposedto)
            # print("resourcespplayer proposedto[1]", resourcespplayer[proposedto[0],:]," ",proposedto[1])
            # print(np.all(resourcespplayer[proposedto[0],:]+proposedto[1]>0)," ",resourcespplayer[proposedto[0],:]+proposedto[1])
            if np.all(resourcespplayer[proposedto[0],:]+proposedto[1]>=0):
                filteredexchanges.append(proposedto)
        # print("checkifpossibleexchange", exchanges, filteredexchanges)
        return filteredexchanges



    def updateResources(self, res):
        self.res = res
        return

    def propose(self, resourcespplayer):

        # choose opponent to suggest exchange first
        order = [self.otherplayer[0], self.otherplayer[1]]
        # print("order", order)

        for rr in [0,1]:
            resourcestomakerecipe = self.res-self.recipes[rr]
            # print("propose ", self.res, self.recipes[rr], resourcestomakerecipe)
            exchanges = []
            if np.sum(np.abs(resourcestomakerecipe))<2: #illegal to give a card
                continue
            else:
                if np.sum(resourcestomakerecipe)==0: # exchange one to one
                    exchanges = [ [order[0], resourcestomakerecipe], [order[1], resourcestomakerecipe]]
                elif np.sum(resourcestomakerecipe)==1: # exchange two to one
                    exchanges = [ [order[0], resourcestomakerecipe], [order[1], resourcestomakerecipe]]
                elif np.sum(resourcestomakerecipe)==-1: # exchange one to two
                    exchanges = [ [order[0], resourcestomakerecipe], [order[1], resourcestomakerecipe]]

        # print("exchanges a", exchanges)
        # only one exchange can be made
        if exchanges:
            exchanges = self.checkifpossibleexchange(resourcespplayer, exchanges)
        # print("exchanges b", exchanges)

        return exchanges

    def answerProposal(self, proposal):

        if self.policy == 'yesman':
            return True

        qualityofproposal = np.sum(proposal[1])

        if self.policy == 'fair':
            if qualityofproposal >= 0:
                return True

        if self.policy == 'profit':
            if qualityofproposal > 0:
                return True

        return False

    def __str__(self):
        return  str(self.id)+" "




In [9]:

class Game:
    def __init__(self, NR, NRPP, NP):# @title Class definition for the agents
        self.NR = NR
        self.NRPP = NRPP
        if np.isscalar(NP):
            self.NP = NP
            playertypes = ['yesman','yesman','yesman']
        else:
            self.NP = len(NP)
            playertypes = NP
        self.recipes = [np.array([2,1], dtype=np.int8), np.array([1,2], dtype=np.int8)]

        self.turn = 0
        self.resourcespplayer = np.zeros((self.NP,self.NR))
        self.points = np.zeros(self.NP)

        self.P = [Player(0,2,self.NP,self.recipes,policy=playertypes[0]),
                  Player(1,2,self.NP,self.recipes,policy=playertypes[1]),
                  Player(2,2,self.NP,self.recipes,policy=playertypes[2])]

        for ii in range(self.NP):
            res = np.random.multinomial(self.NRPP,np.array([1,2])/3)
            self.resourcespplayer[ii,:] = res

    def playturn(self):
        genres = np.random.multinomial(self.NRPP,np.array([1,2])/3)
        self.resourcespplayer[self.turn,:] += genres
        self.P[self.turn].updateResources(self.resourcespplayer[self.turn,:])
        # print("turn > ", self.turn, genres)

        exchanges = self.P[self.turn].propose(self.resourcespplayer)
        # print("propose > ", exchanges)

        for ii in range(len(exchanges)):
            if exchanges[ii][0] == self.turn:
                continue
            elif self.P[exchanges[ii][0]].answerProposal( exchanges[ii] ):
                #process exchanges
                # print("exchange ", exchanges[ii], "accepted")
                self.resourcespplayer[exchanges[ii][0],:] += exchanges[ii][1]
                self.resourcespplayer[self.turn,:] -= exchanges[ii][1]
                self.P[exchanges[ii][0]].updateResources( self.resourcespplayer[exchanges[ii][0],:] )
                self.P[self.turn].updateResources( self.resourcespplayer[self.turn,:] )
                break
            else:
                pass

        self.resourcespplayer[self.turn,:] = np.minimum( self.resourcespplayer[self.turn,:], 5)

        return

    def nextturn(self):

        #build
        m0 = np.min(self.resourcespplayer[self.turn,:]//self.recipes[0])
        self.resourcespplayer[self.turn,:] -= self.recipes[0]*m0
        self.points[self.turn] += 2*m0

        m1 = np.min(self.resourcespplayer[self.turn,:]//self.recipes[1])
        self.resourcespplayer[self.turn,:] -= self.recipes[1]*m1
        self.points[self.turn] += m1
        # print("build ", m0, " recipe 0", m1, " recipe  1")

        self.turn = (self.turn+1)%self.NP
        return

    def __str__(self):
        ret = str(self.turn)+"\n"
        ret += str(self.resourcespplayer)+"\n"
        ret += "points> " + str(self.points)
        return ret


The following cell shows how to run a game.

In [10]:
np.random.seed(1498)
g = Game(2,1,['profit','profit','profit'])
# 'yesman' - always accepts proposals
# 'fair'   - only fairorprofit
# 'profit' - only profit

# print(g)
for tt in range(1000):
    g.playturn()
    g.nextturn()
print(g)

1
[[0. 5.]
 [0. 4.]
 [0. 4.]]
points> [107. 112. 108.]


The following cell runs several experiments with different combinations of type of players.

In [11]:

profiles = [['profit','profit','profit'],
         ['yesman','yesman','yesman'],
         ['fair','fair','fair'],
         ['profit','yesman','yesman'],
         ['profit','fair','fair'],
         ['fair','yesman','yesman'],
         ['fair','profit','profit'],
         ['yesman','profit','profit'],
         ['yesman','fair','fair']         ]

print("type of agents,     points per player,        mean points")
for pp in profiles:
    g = Game(2,1,pp)
    for tt in range(10000):
        g.playturn()
        g.nextturn()
    
    print(pp, g.points, np.round(np.mean(g.points),0))

type of agents,     points per player,        mean points
['profit', 'profit', 'profit'] [1101. 1106. 1092.] 1100.0
['yesman', 'yesman', 'yesman'] [1072. 1104. 1100.] 1092.0
['fair', 'fair', 'fair'] [1127. 1091. 1094.] 1104.0
['profit', 'yesman', 'yesman'] [1087. 1142. 1137.] 1122.0
['profit', 'fair', 'fair'] [1068. 1137. 1129.] 1111.0
['fair', 'yesman', 'yesman'] [1135. 1090. 1094.] 1106.0
['fair', 'profit', 'profit'] [1220. 1071. 1095.] 1129.0
['yesman', 'profit', 'profit'] [1206. 1088. 1081.] 1125.0
['yesman', 'fair', 'fair'] [1103. 1100. 1127.] 1110.0


**Activities**

Analyse the different behaviours. Are the results according to your expactations?

What is the best strategy?

Suggest, and implement ways to improve the mean points.

Suggest a new agent that is able to win against all the other types.

